In [ ]:
import bs4
import urllib3
import time
import pandas as pd
import warnings
import json
warnings.simplefilter(action = 'ignore')

In [ ]:
http = urllib3.PoolManager()

#Años a extraer
w_anio_dde = 2017
w_anio_hta = 2017

#meses a extraer
w_mes_dde = 1
w_mes_hta = 12

#paginas de resultados
pagina_dde = 1
pagina_hta = 20

#factor para saltear paginas de resultados para no llevarse solo lo de ppio de mes...
cada_n_paginas = 1

search_pages = [#["https://buscar.lanacion.com.ar/economia/c-Econom%C3%ADa/date-","ECO"],
                #["https://buscar.lanacion.com.ar/politica/c-Pol%C3%ADtica/date-","POL"],
                ["https://buscar.lanacion.com.ar/turismo/c-Turismo/date-","TUR"]]



search_page = "https://buscar.lanacion.com.ar/Opinion/sort-old/date-"
urls = []
for search_class in search_pages:
    search_page = search_class[0]
    clase = search_class[1]
  #  for year in range(w_anio_dde,w_anio_hta+1):   #1997 a 2018
    year = 2018    
    for month in range(w_mes_dde,w_mes_hta+1):     #1 a 13
        for rpage in range(pagina_dde,pagina_hta):   # 1 a 4
            try:
                page = http.request('GET', search_page+str(year)+str(month).zfill(2)+"01,"+
                                    str(year)+str(month).zfill(2)+"28/sort-old/page-"+
                                    str(rpage*cada_n_paginas))
                soup = bs4.BeautifulSoup(page.data, "html.parser")
                first = soup.find("li", {"class": "floatFix notas"})
                urls.append([first.a["href"],clase])
                nsib = first.findNextSibling()
                print(year,month, rpage,len(urls)," ", flush=True)
                while not nsib is None:
                    urls.append([nsib.a["href"],clase])
                    nsib = nsib.findNextSibling()
            except:
                print("skipped", year, month, rpage,clase, search_page+str(year)+str(month).zfill(2)+"01,"+
                                    str(year)+str(month).zfill(2)+"28/sort-old/page-"+
                                    str(rpage*cada_n_paginas))
                continue
        pd.DataFrame(urls).to_csv("./validacion_tp/lanacion_valid_url2.csv")

In [ ]:
urls = pd.read_csv("./validacion_tp/lanacion_valid_url2.csv")

In [ ]:
#prev_items = pd.read_csv("./TP1_articulos.csv")

In [ ]:
#len(prev_items)

In [ ]:
skipped = 0
urls = pd.read_csv("./validacion_tp/lanacion_valid_url2.csv")
news_items = []
for ind,url in urls.iterrows():
    if ind > 0:
        page = http.request('GET', url[1])
        soup = bs4.BeautifulSoup(page.data, "html.parser")
        try:
            titulo1 = soup.find("h1", {"class": "titulo"}).text
            metadatos = json.loads(soup.find('script', type='application/ld+json').text)
            titulo = metadatos["headline"], 
            seccion = metadatos["articleSection"]
            fecha = metadatos["dateCreated"]
            autor = metadatos["creator"]
            tags = metadatos["keywords"]
            first = soup.find("p", {"class": "capital"})
            nsib = first.findNextSibling()
            article = first.string
            if article is None: article = first.text
            while not nsib is None:
                if not nsib.string is None:
                    article = article + nsib.string
                nsib = nsib.findNextSibling()
            news_items.append([url[1],titulo1, titulo, seccion, fecha, autor, tags, article])
            if (ind/1 == int(ind/1)): 
                print(ind, titulo1, titulo, seccion, fecha, autor, tags,url[1])
                pd.DataFrame(news_items, columns = ["url","titulo1", "titulo", "seccion", "fecha", "autor", "tags", "texto"]).to_csv("./validacion_tp/validacion_lanacion2.csv", index_label = "ID")
        except:
            skipped +=1
            print("Skipped ALL: ",skipped, url[1] )
            continue   
pd.DataFrame(news_items,columns = ["url","titulo1", "titulo", "seccion", "fecha", "autor", "tags", "texto"]).to_csv("./validacion_tp/validacion_lanacion2.csv", index_label = "ID")
